In [1]:
import torch.nn as nn
import torch
import copy
import math
import torch.nn.functional as F

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
print(device)

cuda


In [4]:
class Embedder(nn.Module):
    def __init__(self, vocab_size, dim):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, dim)

    def forward(self, x):
        return self.embed(x)

In [5]:
class PositionalEncoder(nn.Module):
    def __init__(self, dim, max_seq_len=300):
        super().__init__()
        self.dim = dim

        pe = torch.zeros(max_seq_len, dim)
        for pos in range(max_seq_len):
            for i in range(0, dim, 2):
                pe[pos, i] = math.sin(pos/ (10000 ** ((2*i)/dim)))
                pe[pos, i+1] = math.cos(pos / (10000 ** ((2* (i+1))/dim)))

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x *math.sqrt(self.dim)
        seq_len = x.size(1)
        x = x + Variable(self.pe[:, :seq_len], requires_grad=False).to(device)
        return x

In [6]:
def attention(q, k, v, d_k, mask=None, dropout=None):
    scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)

    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)

    scores = F.softmax(scores, dim=-1)

    if dropout is not None:
        scores = dropout(scores)

    output = torch.matmul(scores, v)
    return output

In [7]:
class MultiHeadAttention(nn.Module):
    def __init__(self, heads, dim, dropout=0.1):
        super().__init__()
        self.dim = dim
        self.dim_head = dim // heads
        self.h = heads
        self.q_linear = nn.Linear(dim, dim)
        self.k_linear = nn.Linear(dim, dim)
        self.v_linear = nn.Linear(dim, dim)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(dim, dim)

    def forward(self, q, k, v, mask=None):
        bs = q.size(0)

        k = self.k_linear(k).view(bs, -1, self.h, self.dim_head)
        q = self.q_linear(q).view(bs, -1, self.h, self.dim_head)
        v = self.v_linear(v).view(bs, -1, self.h, self.dim_head)

        k = k.transpose(1, 2)
        q = q.transpose(1, 2)
        v = v.transpose(1, 2)

        scores = attention(q, k, v, self.dim_head, mask, self.dropout)

        concat = scores.transpose(1, 2).contiguous().view(bs, -1, self.dim)
        output = self.out(concat)
        return output

In [8]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=1024, dropout = 0.1):
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)
    def forward(self, x):
        x = self.dropout(F.relu(self.linear_1(x)))
        x = self.linear_2(x)
        return x

In [9]:
class Norm(nn.Module):
    def __init__(self, d_model, eps = 1e-6):
        super().__init__()

        self.size = d_model
        self.alpha = nn.Parameter(torch.ones(self.size))
        self.bias = nn.Parameter(torch.zeros(self.size))
        self.eps = eps
    def forward(self, x):
        norm = self.alpha * (x - x.mean(dim=-1, keepdim=True)) \
        / (x.std(dim=-1, keepdim=True) + self.eps) + self.bias
        return norm

In [10]:

class EncoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout = 0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.attn = MultiHeadAttention(heads, d_model)
        self.ff = FeedForward(d_model)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)

    def forward(self, x, mask):
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn(x2,x2,x2,mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.ff(x2))
        return x


In [11]:

class DecoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout=0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.norm_3 = Norm(d_model)

        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        self.dropout_3 = nn.Dropout(dropout)

        self.attn_1 = MultiHeadAttention(heads, d_model)
        self.attn_2 = MultiHeadAttention(heads, d_model)
        self.ff = FeedForward(d_model).cuda()

    def forward(self, x, e_outputs, src_mask, trg_mask):
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn_1(x2, x2, x2, trg_mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.attn_2(x2, e_outputs, e_outputs,
        src_mask))
        x2 = self.norm_3(x)
        x = x + self.dropout_3(self.ff(x2))
        return x

def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

In [12]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.layers = get_clones(EncoderLayer(d_model, heads), N)
        self.norm = Norm(d_model)

    def forward(self, src, mask):
        x = self.embed(src)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, mask)
        return self.norm(x)

class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.layers = get_clones(DecoderLayer(d_model, heads), N)
        self.norm = Norm(d_model)
    def forward(self, trg, e_outputs, src_mask, trg_mask):
        x = self.embed(trg)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, e_outputs, src_mask, trg_mask)
        return self.norm(x)

In [13]:
class Transformer(nn.Module):
    def __init__(self, src_vocab, trg_vocab, d_model, N, heads):
        super().__init__()
        self.encoder = Encoder(src_vocab, d_model, N, heads)
        self.decoder = Decoder(trg_vocab, d_model, N, heads)
        self.out = nn.Linear(d_model, trg_vocab)
    def forward(self, src, trg, src_mask, trg_mask):
        e_outputs = self.encoder(src, src_mask)
        d_output = self.decoder(trg, e_outputs, src_mask, trg_mask)
        output = self.out(d_output)
        return output


In [14]:
import spacy
import re
class tokenize(object):

    def __init__(self, lang):
        self.nlp = spacy.load(lang)

    def tokenizer(self, sentence):
        sentence = re.sub(
        r"[\*\"“”\n\\…\+\-\/\=\(\)‘•:\[\]\|’\!;]", " ", str(sentence))
        sentence = re.sub(r"[ ]+", " ", sentence)
        sentence = re.sub(r"\!+", "!", sentence)
        sentence = re.sub(r"\,+", ",", sentence)
        sentence = re.sub(r"\?+", "?", sentence)
        sentence = sentence.lower()
        return [tok.text for tok in self.nlp.tokenizer(sentence) if tok.text != " "]

In [15]:
!pip install transformers datasets sentencepiece


In [16]:
from datasets import load_dataset

dataset = load_dataset("wmt14", "de-en", split="train[:1%]")  # đức ↔ anh
print(dataset[0])




/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'translation': {'de': 'Wiederaufnahme der Sitzungsperiode', 'en': 'Resumption of the session'}}


In [17]:

test_data  = load_dataset("wmt14", "de-en", split="test")

train_data = load_dataset("wmt14", "de-en", split="train[:2%]")
val_data   = load_dataset("wmt14", "de-en", split="validation[:2%]")
print(train_data[0])  # xem sample từ train
print(val_data[0])    # sample từ validation
print(test_data[0])   # sample từ test


{'translation': {'de': 'Wiederaufnahme der Sitzungsperiode', 'en': 'Resumption of the session'}}
{'translation': {'de': 'Eine republikanische Strategie, um der Wiederwahl von Obama entgegenzutreten', 'en': 'A Republican strategy to counter the re-election of Obama'}}
{'translation': {'de': 'Gutach: Noch mehr Sicherheit für Fußgänger', 'en': 'Gutach: Increased safety for pedestrians'}}


In [18]:
!pip install -q spacy
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 138.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 124.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [19]:
from collections import Counter, OrderedDict
from tqdm import tqdm
import spacy
import re


class tokenize(object):
    def __init__(self, lang_model_name):
        # Ví dụ: "de_core_news_sm" hoặc "en_core_web_sm"
        self.nlp = spacy.load(lang_model_name)

    def clean_text(self, sentence: str) -> str:
        # Làm sạch giống code gốc của bạn
        sentence = re.sub(r"[\*\"“”\n\\…\+\-\/\=\(\)‘•:\[\]\|’\!;]", " ", str(sentence))
        sentence = re.sub(r"[ ]+", " ", sentence)
        sentence = re.sub(r"\!+", "!", sentence)
        sentence = re.sub(r"\,+", ",", sentence)
        sentence = re.sub(r"\?+", "?", sentence)
        sentence = sentence.lower().strip()
        return sentence

    def tokenizer(self, sentence: str):
        sent = self.clean_text(sentence)
        return [tok.text for tok in self.nlp.tokenizer(sent) if tok.text.strip() != ""]

de_tokenizer = tokenize("de_core_news_sm")
en_tokenizer = tokenize("en_core_web_sm")


class CustomVocab:
    def __init__(self, counter, min_freq=1, specials=('<unk>',)):
        print("Đang khởi tạo vocab...")

        self.itos = list(specials)  # index -> token
        self.stoi = OrderedDict((tok, i) for i, tok in enumerate(self.itos))

        self.unk_index = self.stoi.get('<unk>', 0)

        sorted_tokens = sorted(counter.items(), key=lambda x: x[1], reverse=True)

        for token, freq in tqdm(sorted_tokens, desc=" -> Lọc và thêm token"):
            if freq >= min_freq and token not in self.stoi:
                self.stoi[token] = len(self.itos)
                self.itos.append(token)

        print("Khởi tạo vocab hoàn tất.")

    def __len__(self):
        return len(self.itos)

    def __getitem__(self, token):
        return self.stoi.get(token, self.unk_index)

    def __call__(self, tokens):
        return [self[token] for token in tokens]

    def set_default_index(self, index):
        self.unk_index = index

def yield_tokens(data_iter, tokenizer_obj, lang_key):
    """
    data_iter: iterable các item HuggingFace (mỗi item có 'translation')
    tokenizer_obj: instance của class tokenize(...) (có .tokenizer)
    lang_key: 'de' hoặc 'en'
    """
    for item in data_iter:
        text = item['translation'][lang_key]
        yield tokenizer_obj.tokenizer(text)

special_tokens = ['<unk>', '<pad>', '<bos>', '<eos>']  # bạn có thể giữ như vậy để dùng cho mô hình của bạn

# Đức (de)
print("Đang xây dựng vocab tiếng Đức (de)...")
print(" -> Đang đếm tần suất (de). Quá trình này có thể mất vài phút...")
de_counter = Counter()
for toks in yield_tokens(tqdm(train_data, desc="Đếm token DE"), de_tokenizer, 'de'):
    de_counter.update(toks)
print(f" -> Đã tìm thấy {len(de_counter)} token duy nhất (de).")

de_vocab = CustomVocab(de_counter, min_freq=2, specials=special_tokens)
de_vocab.set_default_index(de_vocab['<unk>'])  # giữ API cũ của bạn

# Anh (en)
print("\nĐang xây dựng vocab tiếng Anh (en)...")
print(" -> Đang đếm tần suất (en). Quá trình này có thể mất vài phút...")
en_counter = Counter()
for toks in yield_tokens(tqdm(train_data, desc="Đếm token EN"), en_tokenizer, 'en'):
    en_counter.update(toks)
print(f" -> Đã tìm thấy {len(en_counter)} token duy nhất (en).")

en_vocab = CustomVocab(en_counter, min_freq=2, specials=special_tokens)
en_vocab.set_default_index(en_vocab['<unk>'])

print("\nHoàn tất xây dựng vocab!")
print(f"Kích thước vocab 'de': {len(de_vocab)}")
print(f"Kích thước vocab 'en': {len(en_vocab)}")

print("\n--- Kiểm tra Vocab ---")
print(f"Index của '<unk>' (de): {de_vocab['<unk>']}")
print(f"Index của '<pad>' (de): {de_vocab['<pad>']}")
print(f"Index của '<bos>' (de): {de_vocab['<bos>']}")
print(f"Index của '<eos>' (de): {de_vocab['<eos>']}")

test_sentence = "Das ist ein einfacher Test!"
test_tokens = de_tokenizer.tokenizer(test_sentence)
print(f"\nCâu test (de): {test_sentence}")
print(f"Tokens: {test_tokens}")
print(f"Indices: {de_vocab(test_tokens)}")
import pickle




Đang xây dựng vocab tiếng Đức (de)...
 -> Đang đếm tần suất (de). Quá trình này có thể mất vài phút...


Đếm token DE: 100%|██████████| 90176/90176 [00:31<00:00, 2879.83it/s]


 -> Đã tìm thấy 66347 token duy nhất (de).
Đang khởi tạo vocab...


 -> Lọc và thêm token: 100%|██████████| 66347/66347 [00:00<00:00, 2670090.36it/s]


Khởi tạo vocab hoàn tất.

Đang xây dựng vocab tiếng Anh (en)...
 -> Đang đếm tần suất (en). Quá trình này có thể mất vài phút...


Đếm token EN: 100%|██████████| 90176/90176 [00:17<00:00, 5154.77it/s]


 -> Đã tìm thấy 26378 token duy nhất (en).
Đang khởi tạo vocab...


 -> Lọc và thêm token: 100%|██████████| 26378/26378 [00:00<00:00, 2363338.98it/s]

Khởi tạo vocab hoàn tất.

Hoàn tất xây dựng vocab!
Kích thước vocab 'de': 35051
Kích thước vocab 'en': 18137

--- Kiểm tra Vocab ---
Index của '<unk>' (de): 0
Index của '<pad>' (de): 1
Index của '<bos>' (de): 2
Index của '<eos>' (de): 3

Câu test (de): Das ist ein einfacher Test!
Tokens: ['das', 'ist', 'ein', 'einfacher', 'test']
Indices: [12, 17, 31, 4455, 6439]


In [20]:
from torch.utils.data import DataLoader
PAD_IDX = de_vocab['<pad>']
BOS_IDX = de_vocab['<bos>']
EOS_IDX = de_vocab['<eos>']

SRC_VOCAB_SIZE = len(de_vocab)
TRG_VOCAB_SIZE = len(en_vocab)
D_MODEL = 256
N_LAYERS = 6
HEADS = 8
DROPOUT = 0.1
BATCH_SIZE = 32

N_EPOCHS = 5
LR = 0.0001

def collate_fn(batch):
    src_batch, trg_batch = [], []
    for item in batch:
        src_sample = item['translation']['de']
        trg_sample = item['translation']['en']

        src_tensor = torch.tensor(
            [BOS_IDX] + de_vocab(de_tokenizer.tokenizer(src_sample)) + [EOS_IDX],
            dtype=torch.long
        )
        trg_tensor = torch.tensor(
            [BOS_IDX] + en_vocab(en_tokenizer.tokenizer(trg_sample)) + [EOS_IDX],
            dtype=torch.long
        )
        src_batch.append(src_tensor)
        trg_batch.append(trg_tensor)

    src_batch_padded = pad_sequence(src_batch, padding_value=PAD_IDX, batch_first=True)
    trg_batch_padded = pad_sequence(trg_batch, padding_value=PAD_IDX, batch_first=True)

    return src_batch_padded, trg_batch_padded

train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

In [21]:
model = Transformer(SRC_VOCAB_SIZE, TRG_VOCAB_SIZE, D_MODEL, N_LAYERS, HEADS).to(device)

def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)
model.apply(initialize_weights)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = torch.optim.Adam(model.parameters(), lr=LR, betas=(0.9, 0.98), eps=1e-9)

In [22]:
def create_pad_mask(seq, pad_idx):
    return (seq != pad_idx).unsqueeze(1).unsqueeze(2).to(device)

def create_look_ahead_mask(seq):
    seq_len = seq.shape[1]
    look_ahead_mask = torch.tril(torch.ones((seq_len, seq_len), device=device)).bool()
    return look_ahead_mask

def create_masks(src, trg, pad_idx):
    src_mask = create_pad_mask(src, pad_idx)
    trg_pad_mask = create_pad_mask(trg, pad_idx)
    trg_look_ahead_mask = create_look_ahead_mask(trg)
    trg_mask = trg_pad_mask & trg_look_ahead_mask.unsqueeze(0).unsqueeze(0)

    return src_mask, trg_mask

In [23]:
def train_epoch(model, loader, optimizer, criterion, pad_idx):
    model.train()
    epoch_loss = 0

    for batch in tqdm(loader, desc="Training"):
        src, trg = batch
        src = src.to(device)
        trg = trg.to(device)

        trg_input = trg[:, :-1]
        trg_output = trg[:, 1:]

        src_mask, trg_mask = create_masks(src, trg_input, pad_idx)

        optimizer.zero_grad()

        output = model(src, trg_input, src_mask, trg_mask)

        output_flat = output.contiguous().view(-1, TRG_VOCAB_SIZE)
        trg_output_flat = trg_output.contiguous().view(-1)

        loss = criterion(output_flat, trg_output_flat)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(loader)

In [24]:
def evaluate_epoch(model, loader, criterion, pad_idx):
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for batch in tqdm(loader, desc="Evaluating"):
            src, trg = batch
            src = src.to(device)
            trg = trg.to(device)

            trg_input = trg[:, :-1]
            trg_output = trg[:, 1:]

            src_mask, trg_mask = create_masks(src, trg_input, pad_idx)

            output = model(src, trg_input, src_mask, trg_mask)

            output_flat = output.contiguous().view(-1, TRG_VOCAB_SIZE)
            trg_output_flat = trg_output.contiguous().view(-1)

            loss = criterion(output_flat, trg_output_flat)
            epoch_loss += loss.item()

    return epoch_loss / len(loader)

In [25]:
import time
from torch.nn.utils.rnn import pad_sequence
from torch.autograd import Variable

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()

    train_loss = train_epoch(model, train_dataloader, optimizer, criterion, PAD_IDX)
    valid_loss = evaluate_epoch(model, val_dataloader, criterion, PAD_IDX)

    end_time = time.time()
    epoch_mins, epoch_secs = divmod(end_time - start_time, 60)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'transformer-wmt14-de-en.pt')

    print(f"\nEpoch: {epoch+1:02} | Time: {epoch_mins:.0f}m {epoch_secs:.0f}s")
    print(f"\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}")
    print(f"\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}")

print("\nTraining hoàn tất!")
print(f"Model tốt nhất đã được lưu vào 'transformer-wmt14-de-en.pt'")

Evaluating: 100%|██████████| 2/2 [00:00<00:00, 31.08it/s]



Epoch: 01 | Time: 6m 27s
	Train Loss: 5.207 | Train PPL: 182.505
	 Val. Loss: 5.386 |  Val. PPL: 218.326


Evaluating: 100%|██████████| 2/2 [00:00<00:00, 32.24it/s]



Epoch: 02 | Time: 6m 22s
	Train Loss: 4.354 | Train PPL:  77.816
	 Val. Loss: 4.953 |  Val. PPL: 141.601


Evaluating: 100%|██████████| 2/2 [00:00<00:00, 29.66it/s]



Epoch: 03 | Time: 6m 23s
	Train Loss: 3.914 | Train PPL:  50.084
	 Val. Loss: 4.610 |  Val. PPL: 100.507


Evaluating: 100%|██████████| 2/2 [00:00<00:00, 32.75it/s]



Epoch: 04 | Time: 6m 21s
	Train Loss: 3.568 | Train PPL:  35.435
	 Val. Loss: 4.333 |  Val. PPL:  76.186


Evaluating: 100%|██████████| 2/2 [00:00<00:00, 21.25it/s]



Epoch: 05 | Time: 6m 24s
	Train Loss: 3.298 | Train PPL:  27.053
	 Val. Loss: 4.132 |  Val. PPL:  62.306

Training hoàn tất!
Model tốt nhất đã được lưu vào 'transformer-wmt14-de-en.pt'


In [29]:
test_data  = load_dataset("wmt14", "de-en", split="test")
import torch
from tqdm import tqdm
from datasets import load_metric # <-- Dùng lại thư viện 'datasets' cũ
import warnings

def translate_sentence(model, sentence, de_vocab, en_vocab, de_tokenizer, device, max_len=100):

    model.eval() # Đảm bảo model ở chế độ eval

    # 1. Tokenize và numericalize câu nguồn (de)
    tokens = de_tokenizer.tokenizer(sentence)
    indices = [de_vocab['<bos>']] + de_vocab(tokens) + [de_vocab['<eos>']]
    src_tensor = torch.LongTensor(indices).unsqueeze(0).to(device)

    # 2. Tạo source mask (chỉ dùng create_pad_mask)
    src_mask = create_pad_mask(src_tensor, de_vocab['<pad>'])

    # 3. Chạy Encoder 1 LẦN DUY NHẤT
    with torch.no_grad():
        e_outputs = model.encoder(src_tensor, src_mask)

    # 4. Vòng lặp tự hồi quy của Decoder
    trg_indices = [en_vocab['<bos>']] # Bắt đầu câu dịch với <bos>

    for i in range(max_len):
        trg_tensor = torch.LongTensor(trg_indices).unsqueeze(0).to(device)

        # 5. Tạo target mask
        trg_mask = create_masks(src_tensor, trg_tensor, de_vocab['<pad>'])[1]

        # 6. Chạy decoder
        with torch.no_grad():
            output = model.decoder(trg_tensor, e_outputs, src_mask, trg_mask)
            pred = model.out(output)

        # 7. Lấy từ cuối cùng model dự đoán (Greedy search)
        pred_token_idx = pred.argmax(2)[:, -1].item()

        trg_indices.append(pred_token_idx) # Thêm từ dự đoán vào chuỗi dịch

        # 8. Dừng nếu model dịch ra <eos>
        if pred_token_idx == en_vocab['<eos>']:
            break

    # 9. Chuyển indices về lại chữ (bỏ <bos> ở đầu)
    trg_tokens = [en_vocab.itos[i] for i in trg_indices[1:]]

    # 10. Ghép lại thành câu, dừng ở <eos> nếu có
    output_sentence = []
    for tok in trg_tokens:
        if tok == '<eos>':
            break
        output_sentence.append(tok)

    return " ".join(output_sentence)

# =============================================================================
# ▼▼▼ CODE TEST CHÍNH (ĐÃ SỬA) ▼▼▼
# =============================================================================

print("--- BẮT ĐẦU TEST (Dùng thư viện 'datasets' cũ) ---")

# --- Bước 1: Tải model (Nếu cần) ---
model.eval() # LUÔN LUÔN set .eval() khi test

# --- Bước 2: Tải phép đo BLEU (Dùng 'load_metric' cũ) ---
print("Đang tải metric BLEU...")
bleu_metric = load_metric("bleu", trust_remote_code=True)
print("Tải metric hoàn tất.")

# --- Bước 3: Chạy dịch trên tập test ---
print("Đang chạy dịch trên tập test (chỉ 2% data)...")
hypotheses = []  # List các *list từ* model dịch
references = []  # List các *list của list từ* dịch mẫu

# Chạy vòng lặp qua test_data
for example in tqdm(test_data):
    src_text = example['translation']['de']
    trg_text = example['translation']['en']

    # Dịch câu (kết quả là string)
    prediction = translate_sentence(model, src_text, de_vocab, en_vocab, de_tokenizer, device)

    # ▼▼▼ THAY ĐỔI QUAN TRỌNG Ở ĐÂY ▼▼▼
    # Tách từ (tokenize) bằng .split() để fix lỗi ValueError
    hypotheses.append(prediction.split())
    references.append([trg_text.split()]) # Cả reference cũng phải .split()
    # ▲▲▲ HẾT THAY ĐỔI ▲▲▲

# --- Bước 4: Tính điểm ---
# Bây giờ `hypotheses` và `references` đã đúng định dạng
print("Đang tính điểm BLEU...")
final_bleu = bleu_metric.compute(predictions=hypotheses, references=references)

print("\n--- HOÀN TẤT TEST ---")
print(f"Lưu ý: Điểm số này dựa trên tập train CHỈ CÓ 2% data.")
print(f"Điểm BLEU trên tập test: {final_bleu['bleu'] * 100:.2f}")

print("\n--- VÍ DỤ BẢN DỊCH ---")
# In ra 3 ví dụ đầu tiên (Ghép lại các từ đã split để in)
for i in range(min(3, len(hypotheses))):
    print(f"\n[{i+1}]")
    print(f"  Nguồn (de): {test_data[i]['translation']['de']}")
    print(f"  Mẫu (en):   {' '.join(references[i][0])}")
    print(f"  Model (en): {' '.join(hypotheses[i])}")


--- BẮT ĐẦU TEST (Dùng thư viện 'datasets' cũ) ---
Đang tải metric BLEU...
Tải metric hoàn tất.
Đang chạy dịch trên tập test (chỉ 2% data)...


100%|██████████| 3003/3003 [14:12<00:00,  3.52it/s]


Đang tính điểm BLEU...

--- HOÀN TẤT TEST ---
Lưu ý: Điểm số này dựa trên tập train CHỈ CÓ 2% data.
Điểm BLEU trên tập test: 1.59

--- VÍ DỤ BẢN DỊCH ---

[1]
  Nguồn (de): Gutach: Noch mehr Sicherheit für Fußgänger
  Mẫu (en):   Gutach: Increased safety for pedestrians
  Model (en): more than more than a reduction in the food safety of the food safety of the food safety

[2]
  Nguồn (de): Sie stehen keine 100 Meter voneinander entfernt: Am Dienstag ist in Gutach die neue B 33-Fußgängerampel am Dorfparkplatz in Betrieb genommen worden - in Sichtweite der älteren Rathausampel.
  Mẫu (en):   They are not even 100 metres apart: On Tuesday, the new B 33 pedestrian lights in Dorfparkplatz in Gutach became operational - within view of the existing Town Hall traffic lights.
  Model (en): they are no longer in a way of <unk> , on monday , the new <unk> of <unk> <unk> <unk> , <unk> <unk> <unk> in the <unk> of the elderly .

[3]
  Nguồn (de): Zwei Anlagen so nah beieinander: Absicht oder Schildb